In [2]:
import os
from dotenv import load_dotenv
import mlflow
from mlflow.models import infer_signature
from mlflow_utils import get_mlflow_experiment


from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import pandas as pd 

load_dotenv()

True

In [3]:
# conectar con mlflow y minio (por http)
mlflow.set_tracking_uri("http://127.0.0.1:5000")

os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://127.0.0.1:9000"
os.environ['AWS_ACCESS_KEY_ID'] = os.getenv('KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.getenv('ACCESS_KEY')

In [4]:
# experiment = get_mlflow_experiment(experiment_name="testing_mlflow")
# print("Name: {}".format(experiment.name))

mlflow.set_experiment("mlflow_tracking")

# with mlflow.start_run(run_name="logging_models", experiment_id=experiment.experiment_id) as run:
with mlflow.start_run(run_name="logging_artifacts13") as run:
    
    X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, random_state=42)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)

    X_train = pd.DataFrame(X_train, columns=["feature_{}".format(i) for i in range(10)])
    X_test = pd.DataFrame(X_test, columns=["feature_{}".format(i) for i in range(10)])
    y_train = pd.DataFrame(y_train, columns=["target"])
    y_test = pd.DataFrame(y_test, columns=["target"])

    rfc = RandomForestClassifier(n_estimators=100, random_state=42)
    rfc.fit(X_train, y_train)
    y_pred = rfc.predict(X_test)
    y_pred = pd.DataFrame(y_pred, columns=["prediction"])

    # infer signature
    model_signature = infer_signature(model_input=X_train, model_output=y_pred)


    # log model 
    mlflow.sklearn.log_model(sk_model=rfc, artifact_path="random_forest_classifier", signature=model_signature)
    
    
    # print info about the run
    print("run_id: {}".format(run.info.run_id))
    print("experiment_id: {}".format(run.info.experiment_id))
    print("status: {}".format(run.info.status))
    print("start_time: {}".format(run.info.start_time))
    print("end_time: {}".format(run.info.end_time))
    print("lifecycle_stage: {}".format(run.info.lifecycle_stage))

/home/oecorrechag/.local/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/oecorrechag/.local/lib/python3.10/site-packages/mlflow/types/utils.py:394: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-

run_id: 48df5495ed084cebb357aca4a81ad468
experiment_id: 3
status: RUNNING
start_time: 1717780365618
end_time: None
lifecycle_stage: active


In [5]:
print('tracking uri:', mlflow.get_tracking_uri())
print('artifact uri:', mlflow.get_artifact_uri())

tracking uri: http://127.0.0.1:5000
artifact uri: s3://mlflow/3/3477a3adc554416a85067fbf1e2d495b/artifacts


In [6]:
mlflow.end_run()

In [7]:
print('ok_')

Ok_
